In [ ]:
import string
import re
import json

import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import mapclassify

from graphly.api_client import SparqlClient

# TODO: 
# 1. simplify geometries!!!
# 2. Join query with swisstopo

# 3. Idea: Electricity prices over time
# 4. Idea: Can we get location of powerplants as linked data?

In [ ]:
ENDPOINT = "https://lindas.admin.ch/query"

sparql = SparqlClient(ENDPOINT)
sparql.add_prefixes({
    "schema": "<http://schema.org/>",
    "cube": "<https://cube.link/>",
    "property": "<https://ld.stadt-zuerich.ch/statistics/property/>",
    "measure": "<https://ld.stadt-zuerich.ch/statistics/measure/>",
    "skos": "<http://www.w3.org/2004/02/skos/core#>",
    "ssz": "<https://ld.stadt-zuerich.ch/statistics/>"
})

In [ ]:
query = """
PREFIX cube: <https://cube.link/>
PREFIX elcom: <https://energy.ld.admin.ch/elcom/electricityprice/dimension/>
PREFIX schema: <http://schema.org/>

SELECT ?municipality_uri ?category ?energy ?grid ?aidfee (?community_fees + ?aidfee as ?taxes) ?fixcosts ?variablecosts 
FROM <https://lindas.admin.ch/elcom/electricityprice>
WHERE {
    <https://energy.ld.admin.ch/elcom/electricityprice/observation/> cube:observation ?observation.
    
    ?observation
      elcom:category/schema:name ?category;
      elcom:municipality ?muni_iri;
      elcom:period "2020"^^<http://www.w3.org/2001/XMLSchema#gYear>;
      elcom:product <https://energy.ld.admin.ch/elcom/electricityprice/product/standard>;
      elcom:fixcosts ?fixcosts;
      elcom:total ?variablecosts;
      elcom:gridusage ?grid;
      elcom:energy ?energy;
      elcom:charge ?community_fees;
      elcom:aidfee ?aidfee.
    
    BIND(IRI(REPLACE(STR(?muni_iri),"https://ld.admin.ch/", "https://ld.geo.admin.ch/boundaries/")) AS ?municipality_uri) .
}
"""

prices = sparql.send_query(query)
prices.head()

In [ ]:
query = """
PREFIX schema: <http://schema.org/>

SELECT DISTINCT ?category ?description
WHERE {
  GRAPH <https://lindas.admin.ch/elcom/electricityprice> {
    
    ?s <https://energy.ld.admin.ch/elcom/electricityprice/dimension/category> ?category_uri.
    ?category_uri schema:name ?category .
    ?category_uri schema:description ?description .
  }
}
ORDER BY ?category
"""
df = sparql.send_query(query)
df.head()

In [ ]:
def extract_consumption(description: str) -> int:
    """
    Extract average electricity consumption from a description.
    Args:
        description:  Category description for electricity prices
        
    Returns: 
        int:          Electricity consumption in kWh/year
    
    """
    
    number_as_string = description.split(" kWh/Jahr")[0]
    return int(number_as_string.translate(str.maketrans('', '', string.punctuation)))

In [ ]:
cat2description = dict(zip(df.category, df.description))
cat2consumption = dict(zip(df.category, [extract_consumption(d) for d in df.description]))

prices["consumption"] = prices[["category"]].replace({"category": cat2consumption})
prices["monthly_bill"] = (prices.consumption*prices.variablecosts/12 + prices.fixcosts)/100
prices.head()

In [ ]:
geosparql = SparqlClient("https://ld.geo.admin.ch/query")

query = """
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX geonames: <http://www.geonames.org/ontology#>
PREFIX schema: <http://schema.org/>
PREFIX geosparql: <http://www.opengis.net/ont/geosparql#>
    
SELECT ?municipality_uri ?municipality ?population ?polygon WHERE {
  
  ?municipality_uri dct:hasVersion ?version ;
                geonames:featureCode geonames:A.ADM3 .
  
  ?version schema:validUntil "2020-12-31"^^<http://www.w3.org/2001/XMLSchema#date>;
           geonames:population ?population ;
           schema:name ?municipality .
  
  ?version geosparql:hasGeometry/geosparql:asWKT ?polygon
  
} 
"""
communes = geosparql.send_query(query)
communes = communes.set_crs(epsg=4326)
communes.head()

In [ ]:
join = pd.merge(communes, prices, how="inner", on="municipality_uri")
join.drop(columns=["municipality_uri", "consumption"], inplace=True)
join.head()

In [ ]:
def plot_prices_heatmap(category, color_palette, variable, legend_label, N):
    
    colors = sns.color_palette(color_palette, n_colors=N).as_hex()
    
    df = join[join["category"] == category]
    df = df.set_index("municipality")

    classifier = mapclassify.NaturalBreaks(y=df[variable], k=N)
    df["buckets"] = df[[variable]].apply(classifier)
    labels = mapclassify.classifiers._get_mpl_labels(classifier, fmt="{:.0f}")
    labels = ["-".join(re.findall(r"\d+", l)) for l in labels]
    bucket2label = dict(zip(range(N), labels))
    df = df.replace({"buckets": bucket2label})

    colormap={bucket2label[i]: color for i, color in enumerate(colors)}

    fig = px.choropleth(df, geojson=json.loads(df.to_json()), locations=df.index, 
                        color="buckets",
                        color_discrete_map=colormap,
                        projection="transverse mercator",
                        hover_name=df.index,
                        hover_data={"buckets": False, variable: ':.2f'},
                        title="Electricity Prices: {}".format(cat2description[category]),
                        labels={"buckets": legend_label})
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"l":0,"b":0})
    fig.update_traces(marker_line_width=0)
    fig.show()
    

In [ ]:
colors="YlGn"
plot_prices_heatmap("H2", colors, "monthly_bill", "average monthly bill [CHF]", 5)

In [ ]:
plot_prices_heatmap("C1", colors, 'monthly_bill', "average monthly bill [CHF]", 5)

In [ ]:
# Extension: Same map for different tariffs (side-to-side)

### Grid costs and population density 
=> which region is the most expensive to maintain
Does that correlate with area?

In [ ]:
join["hectares"] = join.to_crs(epsg=3035).area/10000 # In hectares
join["population_density"] = join["population"]/join["hectares"]
join.head()

In [ ]:
plot_prices_heatmap("C1", "YlOrRd", 'population_density', "Inhabitants per ha", 6)

In [ ]:
plot_prices_heatmap("C1", "YlOrRd", 'grid', "Grid usage [Rp/kWh]", 6)

In [ ]:
# Relationship between grid costs, and population density
dff = join[join.category == "C1"]

fig = px.scatter(dff, y="grid", x="population_density", hover_data=["municipality"],
                labels={
                     "population_density": "Inbahitants per ha",
                     "grid": "Grid costs per kWh"})
fig.show()

In [ ]:
# Idea: add a map that shows these communes with 0 grid costs
join = join.assign(pays_for_grid=lambda x: x.grid!=0)
join.head()

In [ ]:
df = join[join.category=="C1"]
df = df.set_index("municipality")
fig = px.choropleth(df, geojson=json.loads(df.to_json()), locations=df.index, 
                    color="pays_for_grid",
                    projection="transverse mercator",
                    hover_name=df.index,
                    #hover_data={"buckets": False},
                    labels={"pays_for_grid": "Paid grid usage"})
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"l":0,"b":0})
fig.update_traces(marker_line_width=0)
fig.show()

In [ ]:
# Idea: these communes do not pay Bundesabgaben (aidfee)
join = join.assign(pays_aidfee=lambda x: x.aidfee!=0)
join = join.assign(pays_energy=lambda x: (x.energy)!=0)

df = join[join.category=="C1"]
df = df.set_index("municipality")
data=json.loads(df.to_json())

In [ ]:
fig = px.choropleth(df, geojson=data, locations=df.index, 
                    color="pays_aidfee",
                    projection="transverse mercator",
                    hover_name=df.index,
                    labels={"pays_aidfee": "Pays aidfee"},
                    color_discrete_map={True: "#00CC96", False: "#AB63FA"}
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"l":0,"b":0})
fig.update_traces(marker_line_width=0)
fig.show()

In [ ]:
df = join[join.category=="C4"]
df = df.set_index("municipality")
data=json.loads(df.to_json())

fig = px.choropleth(df, geojson=data, locations=df.index, 
                    color="pays_energy",
                    projection="transverse mercator",
                    hover_name=df.index,
                    labels={"pays_energy": "Pays energy"}
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"l":0,"b":0}, title=cat2description["C4"])
fig.update_traces(marker_line_width=0)
fig.show()

In [ ]:
# Idea: Zefix
# find companies registered in this region (and hence entitiled to free elencticity)
# AGs in Sufers: https://s.zazuko.com/y3SDQ
    
df[(~df.pays_energy)]

In [ ]:
cat2description

In [ ]:
# Idea 3: Distance centroid-powerplant. Correlate distance with the grid usage prices.
# the further it is, the more grid infrastructure is needed, the more expensive it should get ?
# Population density (ppl/area) + grid prices => compare

name = "viridis"
print(sns.color_palette(name, n_colors=15).as_hex())
sns.color_palette(name, n_colors=10).as_hex()

## Join DFs

In [ ]:
query = """
PREFIX cube: <https://cube.link/>
PREFIX elcom: <https://energy.ld.admin.ch/elcom/electricityprice/dimension/>
PREFIX schema: <http://schema.org/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX geonames: <http://www.geonames.org/ontology#>
PREFIX geosparql: <http://www.opengis.net/ont/geosparql#>

SELECT ?municipality ?category ?energy ?grid (?community_fees + ?aidfee as ?taxes) ?fixcosts ?variablecosts 
FROM <https://lindas.admin.ch/elcom/electricityprice>
WHERE {
    <https://energy.ld.admin.ch/elcom/electricityprice/observation/> cube:observation ?observation.
    
    ?observation
      elcom:category <https://energy.ld.admin.ch/elcom/electricityprice/category/H1>;
      elcom:municipality ?muni_iri;
      elcom:period "2020"^^<http://www.w3.org/2001/XMLSchema#gYear>;
      elcom:product <https://energy.ld.admin.ch/elcom/electricityprice/product/standard>;
      elcom:fixcosts ?fixcosts;
      elcom:total ?variablecosts;
      elcom:gridusage ?grid;
      elcom:energy ?energy;
      elcom:charge ?community_fees;
      elcom:aidfee ?aidfee.
    
    BIND(IRI(REPLACE(STR(?muni_iri),"https://ld.admin.ch/", "https://ld.geo.admin.ch/boundaries/")) AS ?municipality) .
  
    {SERVICE <https://ld.geo.admin.ch/query> {
      SELECT ?municipality ?name ?population ?polygon WHERE {
        
        ?municipality dct:hasVersion ?version ;
                      geonames:featureCode geonames:A.ADM3 .
        
        ?version schema:validUntil "2020-12-31"^^<http://www.w3.org/2001/XMLSchema#date>;
             geonames:population ?population ;
             schema:name ?name .
        
        ?version geosparql:hasGeometry/geosparql:asWKT ?polygon
    }
    }}
}
LIMIT 500
"""

df = sparql.send_query(query)
df.head()